#Author: Srikaran Elakurthy

# Desciption
> Import the oversampled train set and test set for using in Naive Bayes model.
Implementing the Multinomial Naive bayes classification on the above dataset.
Performing Cross validation with kfolds =5 and extract average accuracy
Performing the naive bayes model on whole train data and test it and storing the results into files

>*Detailed description is written for many parts of the code below. Please read through for the same.

#Inputs and Outputs

Inputs:
> oversampling_test.csv - The oversampled distribution of records of other genres and undersampled records from the Rock genre. This is only used as the test set.

> oversampling_train.csv - The oversampled distribution of records of other genres and undersampled records from the Rock genre. This is only used as the train set.

Outputs:
> Naiveovrsamp_results.txt - Cross Validation results written into a file
> finalNaiveoversamp_results.txt - Results obtained on the test set after final train.
> final_NaiveBayesovrsam - The final trained model written into a pickle file.
> finalresNaivovrsam.csv - The actual vs predicted genres CSV. 

> *The inputs to the program must be in the same folder as the script.


INPUT: The input to the program are oversampling_test.csv and oversampling_train.csv
OUTPUT:
report:
              precision    recall  f1-score   support

       Other       0.85      0.16      0.27      5753
        Rock       0.98      0.19      0.32      5725
     Country       0.95      0.92      0.93      5694
     Electronic    0.72      0.78      0.75      5708
       Indie       0.99      0.60      0.75      5448
       Metal       0.70      0.18      0.28      4340
     Hip-Hop       0.92      0.37      0.53      5650
         Pop       0.81      0.42      0.55      5433
        Folk       0.50      0.11      0.18      7014
         R&B       0.94      0.58      0.72      5381
        Jazz       0.25      0.95      0.40     10111

    accuracy                          0.51     66257
    macro avg     0.78      0.48      0.52     66257
    weighted avg  0.74      0.51      0.50     66257

Importing the python packages and reading train and test csv files

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk import pos_tag
import nltk
#nltk.download('averaged_perceptron_tagger')
#from langdetect import detect
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
#import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.externals import joblib 
from sklearn.preprocessing import Normalizer
from scipy.sparse import hstack
from sklearn.metrics import classification_report
from statistics import mean
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
testdf=pd.read_csv("oversampling_test.csv")
traindf=pd.read_csv("oversampling_train.csv")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
traindf.head()

,Unnamed: 0,lyrics,genre
0,253258,a kay a kay jordan de shoe pairan vich kaali h...,Other
1,99993,i remeb when i saw you at the diner s yet my h...,Rock
2,267744,you left the water run when you left me here b...,Country
3,132459,thing that happen when you fall asleep when yo...,Electronic
4,312277,oh my god my heart s as heavi as a stone and w...,Indie


Dropping the unnamed columns which was created when importing the csv file

In [ ]:
traindf.drop(traindf.columns[traindf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

testdf.drop(testdf.columns[testdf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)




In [ ]:
lyrics=traindf['lyrics']
genre=traindf['genre']

Performing cross validation with 5 splits and implementing tfidf vectorization inside the cross validation to avoid any data leakage.
TFidf Vectorization will consider parameters stop_words= english specifying that it will be removing any english words and says to consider both unigrams and bigrams.
We are taking Multinomial Naive bayes model specifying we are seeking a multi class problem and alpha =1 specifying smoothing is allowed. 
Storing the models in pickle files using joblib and results with actual and predicted values into a csv file.
Storing the accuracy of every cross validation split into a list.

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=0)
i=0
naivrep=[]
naivscore=[]
for train_index, test_index in skf.split(lyrics, genre):
    print(train_index)
    print(test_index)
    x_train1, x_test1 = lyrics.iloc[train_index], lyrics.iloc[test_index]
    y_train, y_test = genre.iloc[train_index], genre.iloc[test_index]
    i=i+1
    tfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
    x_train = tfidf.fit_transform(x_train1)
    x_test = tfidf.transform(x_test1)
    print("tfidf"+str(i))
    
    clf = MultinomialNB(alpha=1.0)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    joblib.dump(clf, 'NaiveBayesovrsam'+str(i)+'.pkl')
    score = accuracy_score(y_test, y_pred)
    rep=classification_report(y_test, y_pred, target_names=traindf.genre.unique())
    naivrep.append(rep)
    dat={'Actual':y_test,'pred':y_pred}
    resdf=pd.DataFrame(dat)
    resdf.to_csv('resNaivebayesovrsam'+str(i)+'.csv')
    naivscore.append(score)
    print(score)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[ 51850  51866  51875 ... 265023 265024 265025]
[    0     1     2 ... 54345 54349 54374]
tfidf1
0.47017318794098784
[     0      1      2 ... 265023 265024 265025]
[ 51850  51866  51875 ... 107644 107658 107681]
tfidf2
0.4693896802188473
[     0      1      2 ... 265023 265024 265025]
[104764 104787 104788 ... 160528 160554 160556]
tfidf3
0.47029525516460713
[     0      1      2 ... 265023 265024 265025]
[158303 158310 158313 ... 213057 213074 213081]
tfidf4
0.4699556645599472
[     0      1      2 ... 213057 213074 213081]
[211342 211344 211365 ... 265023 265024 265025]
tfidf5
0.4694085463635506


Creating a file to write our classification report for our 5 results of Cross validation models and computing the average accuracy and writing them into the file.
The classification report is computed by using the results csv files containing the actual and predicted values of ech split.

In [ ]:
f=open("Naiveovrsamp_results.txt","a")

for i in range(1,6):
    f.write("\n report"+str(i)+":\n")
    dfr=pd.read_csv("resNaivebayesovrsam"+str(i)+".csv")
    dfr.drop(dfr.columns[dfr.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    f.write(classification_report(dfr.Actual, dfr.pred, target_names=traindf.genre.unique()))
f.write("\nThe average score for this model is")

f.close()

Declaring the naive bayes model with alpha =1 specifying smoothing is allowed.

In [ ]:
finalmod = MultinomialNB(alpha=1.0)

Perform tfidf vectorization to extraxt tfidf matrix on whole training data and use the fitted vectorizer to transform the test data to tfidf matrix.

Tfidf matrix considers:
*   Removing stop words
*   Considering both unigrams and bigrams

In [ ]:
tfidf = TfidfVectorizer(stop_words="english",ngram_range=(1,2))
trainvec = tfidf.fit_transform(traindf['lyrics'])
testvec = tfidf.transform(testdf['lyrics'])

Fitting the model with train tfidf matrix and train target variable(genre's)

In [ ]:
finalmod.fit(trainvec, traindf['genre'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Predicting the test data using test tfidf matrix

In [ ]:
y_pred = finalmod.predict(testvec)

Computing the classification report and writing the results into a file.

In [ ]:
f=open("finalNaiveoversamp_results.txt","a")
f.write("\n report:\n")
f.write(classification_report(testdf['genre'], y_pred, target_names=traindf.genre.unique()))
f.close()

Storing the results into a csv file with actual vs predicted 

In [ ]:
dat={'Actual':testdf['genre'],'pred':y_pred}
resdf=pd.DataFrame(dat)
resdf.to_csv('finalresNaivovrsam.csv')

Storing the model using joblib into. a pickle file

In [ ]:
joblib.dump(finalmod, 'final_NaiveBayesovrsam.pkl')

['/content/drive/My Drive/OverSampling/Final_models/final_NaiveBayesovrsam.pkl']